In [259]:
# ─── 기본 설정 ────────────────────────────────
import os
import io
import json
import base64
from datetime import datetime
from dotenv import load_dotenv

# ─── 이미지 및 EXIF 처리 ──────────────────────
from PIL import Image

# ─── OpenAI 및 LangChain ─────────────────────
from openai import OpenAI

# LangChain 관련 모듈
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage

# textwrap 모듈
import textwrap


In [260]:
SCALEDOWN = 6
OPENAI_TEMPERATURE = 0.5
MAX_TOKENS = 500

In [261]:
# 환경 변수 로드 (.env에 OPENAI_API_KEY 포함되어 있어야 함)
load_dotenv()
client = OpenAI()
# chat_model = ChatOpenAI(temperature=0.9)
chat_model = ChatOpenAI(temperature=OPENAI_TEMPERATURE)


# 이미지 압축 함수 (base64 변환 포함)
def compress_image(image_path, scaledown=SCALEDOWN):
    img = Image.open(image_path)
    new_size = (int(img.width / scaledown), int(img.height / scaledown))
    # img = img.resize(new_size)
    img = img.resize(new_size, resample=Image.LANCZOS)
    img = img.convert("RGB")
    buffer = io.BytesIO()
    img.save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")

In [262]:
def image_to_caption(image_path, event_data):
    b64_img = compress_image(image_path)
    system_prompt = textwrap.dedent(
        f"""\ 
        너는 사진 속 상황을 묘사하는 이미지 캡션 전문가야.

        이 사진을 보고 아래 조건을 지키면서 캡션을 작성해줘:

        [작성 조건]
        - 사진 속 키워드: {', '.join(event_data.get("keywords", ["키워드 없음"]))}
        - 사물, 인물, 배경, 상황을 자연스럽고 사실적으로 묘사해
        - 문장은 문법적으로 완결된 자연스러운 문장으로 작성하고, '같다', '인 듯하다'와 같은 불확실한 표현은 피해
        - 사진에서 보이는 것들을 명확하게 묘사해

        [금지 사항 ❌]
        - 사진 속 텍스트나 글자에 대한 언급
        - 사진이 회전되어 있다는 기술적 정보
        - 날짜, 시간, 계절 등을 직접적으로 언급하거나 추측하지 마
        """
    )

    print(f"\n{image_path}의 캡션 생성 시작...")
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{b64_img}"},
                    }
                ],
            },
        ],
        max_tokens=MAX_TOKENS,
    )
    caption = response.choices[0].message.content
    print(f"생성된 캡션: {caption}")
    return caption


def process_event(event):
    captions = []
    for image_path in event.get("images", []):
        caption = image_to_caption(image_path, event)
        captions.append(caption)

    return {
        "event_id": event.get("id", 0),
        "captions": captions,
        "place": event.get("place", "알 수 없는 장소"),
        "emotion": event.get("emotion", "알 수 없는 감정"),
        "keywords": event.get("keywords", ["키워드 없음"]),
        "start_time": event.get("start_time", "시간 정보 없음"),
    }

In [263]:
# 일상 사건 정의 (하드코딩)
events = [
    {
        "id": 1,
        "images": [
            os.path.join("test", f)
            for f in os.listdir("test")
            if f.startswith("68") or f.startswith("69")
        ],
        "start_time": "14:00",
        "place": "춘천 산토리니 카페",
        "emotion": "행복",
        "keywords": ["춘천", "카페", "포토존"],
    },
    {
        "id": 2,
        "images": [os.path.join("test", f) for f in ["70.jpg", "71.jpg", "78.jpg"]],
        "start_time": "16:00",
        "place": "홍천 알파카월드",
        "emotion": "행복",
        "keywords": ["알파카", "자연", "평화"],
    },
    {
        "id": 3,
        "images": [os.path.join("test", "72.jpg")],
        "start_time": "16:00",
        "place": "홍천 알파카월드",
        "emotion": "행복",
        "keywords": ["알파카월드", "토끼"],
    },
    {
        "id": 6,
        "images": [os.path.join("test", f) for f in ["76.jpg", "77.jpg"]],
        "start_time": "16:40",
        "place": "홍천 알파카월드",
        "emotion": "행복",
        "keywords": ["알파카월드", "작은새", "앵무"],
    },
]

# 각 일상 사건 처리
event_captions = []
for event in events:
    processed_event = process_event(event)
    event_captions.append(processed_event)

# ✅ 일기 작성 프롬프트 개선
diary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            textwrap.dedent(
                """\
                너는 사실적이고 담백한 어조로 하루를 정리하는 일기 작가야.  
                감정에 치우치지 않고, 사진에 담긴 상황과 분위기, 그날의 감정을 자연스럽게 설명해줘.

                [작성 조건]
                - 일상 사건들의 시각적 묘사(captions)와 함께 장소, 감정, 키워드 정보를 참고해서 너무 감성적이지 않게, 담백하고 솔직한 반말 일기체로 작성해줘.
                - 모든 문장은 한국어 '~다'로 끝나는 형식을 지켜줘.
                - 시간은 굳이 정확히 말하지 않아도 되고, 하루 일상의 흐름대로 써줘.
                - 단순 나열이 아닌 하나의 흐름으로, 의미 있는 하루처럼 정리해줘.
                - 문장이 도중에 끊기지 않고 매끄럽게 이어져야 해.
                - 글 길이는 300~500자 정도로 맞춰줘.
                """
            ),
        ),
        (
            "human",
            textwrap.dedent(
                """\
                다음은 오늘 하루 동안 있었던 일상 사건들의 정보야. 이걸 참고해서 일기 한 단락을 써줘.

                [일상 사건 정보]
                {events}
                """
            ),
        ),
    ]
)


# 일상 사건 정보를 문자열로 변환
events_str = "\n".join(
    [
        f"""일상 사건 {event.get('event_id', 0)}:
- 시간: {event.get('start_time', '시간 정보 없음')}
- 장소: {event.get('place', '알 수 없는 장소')}
- 감정: {event.get('emotion', '알 수 없는 감정')}
- 키워드: {', '.join(event.get('keywords', ['키워드 없음']))}
- 일상 사건 요약: {event.get('combined_caption', '일상 사건 요약 없음')}"""
        for event in event_captions
    ]
)

# 일기 생성
messages = diary_prompt.format_messages(events=events_str)
diary = chat_model.invoke(messages).content

# 결과 출력
print("\n\n전체 이미지 기반 하루 요약 일기")
print(diary)


test\68.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 춘천의 한 카페의 포토존을 보여줍니다. 설치된 건축 구조물은 높은 곳에 위치해 있으며, 하얀색으로 깔끔하게 마무리되어 있습니다. 이 구조물은 전통적인 아치형 디자인을 현대적으로 해석한 것 같으며, 정상에는 작은 장식적인 요소가 보입니다. 배경으로는 맑은 하늘이 펼쳐져 있으며, 햇빛이 구조물을 밝게 비추고 있어 사진 촬영 장소로서 매력적인 환경을 제공합니다.

test\69.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 춘천의 한 카페에서 찍은 것으로 보입니다. 사진에는 잘 구워진 황금빛 크로와상 하나가 흰색 접시에 담겨 있으며, 옆에는 나무 포크와 나이프가 놓여 있습니다. 둥근 나무 테이블 위에는 두 개의 파란 무늬가 있는 종이컵에 담긴 커피가 보이는데, 한 컵은 카푸치노로 보이며 위에 초콜릿 소스로 격자 무늬 장식이 되어 있고, 다른 한 컵은 라떼로 보입니다. 전체적인 배치와 조화로운 컬러가 포토존으로서의 매력을 더해주는 카페 분위기를 잘 나타내고 있습니다.

test\70.jpg의 캡션 생성 시작...
생성된 캡션: 알파카들이 자연스러운 환경에서 평화롭게 지내는 모습이 담긴 이 사진은 몇몇 알파카들이 나무 울타리 안에서 구경오는 방문객들을 바라보고 있는 장면을 보여줍니다. 흙길과 녹색의 나무들이 배경에 있어 자연의 느낌을 더해줍니다. 알파카 중 하나는 특히 카메라를 향해 고개를 들고 있는데, 그 모습이 귀엽고 친근감을 줍니다.

test\71.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 대형 알파카 캐릭터 조형물이 마음을 표현하는 하트 모양의 물체를 들고 있는 모습을 보여줍니다. 이 밝은 표정의 알파카는 자연 속에서 평화로움과 친근함을 느끼게 하며, 배경에는 푸른 하늘과 조그만 건물이 보여 자연스러운 환경 속에서 조화롭게 자리잡고 있습니다.

test\78.jpg의 캡션 생성 시작...
생성된 캡션: 이 평화로운 자연 속에 위치한 알파카 목장에서는 검은색과 갈색, 흰

In [264]:
# 일상 사건 정의 (하드코딩)
events = [
    {
        "id": 7,
        "images": [os.path.join("test2", f"{i}.jpg") for i in range(82, 93)],
        "start_time": "15:00",
        "place": "동두천 니지모리 스튜디오",
        "emotion": "행복",
        "keywords": ["일본마을", "아빠", "누나"],
    }
]

# 각 일상 사건 처리
event_captions = []
for event in events:
    processed_event = process_event(event)
    event_captions.append(processed_event)

# ✅ 일기 작성 프롬프트 개선
diary_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            textwrap.dedent(
                """\
                너는 사실적이고 담백한 어조로 하루를 정리하는 일기 작가야.  
                감정에 치우치지 않고, 사진에 담긴 상황과 분위기, 그날의 감정을 자연스럽게 설명해줘.

                [작성 조건]
                - 일상 사건들의 시각적 묘사(captions)와 함께 장소, 감정, 키워드 정보를 참고해서 너무 감성적이지 않게, 담백하고 솔직한 반말 일기체로 작성해줘.
                - 모든 문장은 한국어 '~다'로 끝나는 형식을 지켜줘.
                - 시간은 굳이 정확히 말하지 않아도 되고, 하루 일상의 흐름대로 써줘.
                - 단순 나열이 아닌 하나의 흐름으로, 의미 있는 하루처럼 정리해줘.
                - 문장이 도중에 끊기지 않고 매끄럽게 이어져야 해.
                - 글 길이는 300~500자 정도로 맞춰줘.
                """
            ),
        ),
        (
            "human",
            textwrap.dedent(
                """\
                다음은 오늘 하루 동안 있었던 일상 사건들의 정보야. 이걸 참고해서 일기 한 단락을 써줘.

                [일상 사건 정보]
                {events}
                """
            ),
        ),
    ]
)


# 일상 사건 정보를 문자열로 변환
events_str = "\n".join(
    [
        f"""일상 사건 {event.get('event_id', 0)}:
- 시간: {event.get('start_time', '시간 정보 없음')}
- 장소: {event.get('place', '알 수 없는 장소')}
- 감정: {event.get('emotion', '알 수 없는 감정')}
- 키워드: {', '.join(event.get('keywords', ['키워드 없음']))}
- 일상 사건 요약: {event.get('combined_caption', '일상 사건 요약 없음')}"""
        for event in event_captions
    ]
)

# 일기 생성
messages = diary_prompt.format_messages(events=events_str)
diary = chat_model.invoke(messages).content

# 결과 출력
print("\n\n전체 이미지 기반 하루 요약 일기")
print(diary)


test2\82.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 일본의 전통적인 마을의 거리 모습을 보여줍니다. 사진 속에는 특징적인 일본식 건물과 큰 원형 표지판이 보입니다. 표지판은 특정 지역을 알리는 역할을 하며, 주변은 잘 가꿔진 나무와 조경이 어우러져 있습니다. 건물의 전통적인 검은색과 노란색 조합이 일본 마을의 고유한 분위기를 풍깁니다.

test2\83.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 일본마을에서 찍은 가족 사진으로 보입니다. 사진에는 미소를 짓고 있는 젊은 남성이 가장 앞에 서 있으며, 그의 뒤로 한 남성(아마도 아버지)과 여성(누나로 추정)이 함께 서 있습니다. 여성은 흰색 롱 패딩을 입고 있고, 남성은 짙은색 점퍼를 착용하였습니다. 모두 따뜻한 겨울 옷차림을 하고 있습니다. 배경에는 전통적인 일본 건축양식의 건물과 크리스마스 장식이 보여 휴일 또는 축제 분위기를 더하고 있습니다. 이들은 즐거운 여행 중인 모습입니다.

test2\84.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 일본 마을의 한 가게 내부를 보여줍니다. 사진의 중앙에는 애니메이션 캐릭터들이 그려진 초상화가 놓여 있으며, 이를 둘러싸고 다양한 일본 문화적 요소가 보입니다. 왼쪽에는 전통적인 일본 가면이 전시되어 있고, 오른쪽에는 빨간색 가면이 있다. 이 사진은 아빠와 누나가 일본 문화를 체험하기 위해 방문한 모습일 수 있으며, 배경에 있는 다양한 소품들이 일본의 전통과 현대 문화를 모두 표현하고 있습니다.

test2\85.jpg의 캡션 생성 시작...
생성된 캡션: 이 사진은 전통적인 일본 마을의 거리에서 찍힌 것으로 보입니다. 배경에는 목조 건물들이 보이며, 이 건물들은 검은색 지붕과 짙은 목재 외벽을 갖추고 있습니다. 거리에는 일본식 등롱이 매달려 있고, 날이 어두워지면서 환한 불빛을 발합니다. 사진의 앞 부분에는 손이 보이는데, 손은 한국 여권과 항공권을 들고 있습니다. 이 항공권은 여행자가 방문하기 위해 사용한 것으로 추정되며,